# Фабрика (factory)

Фабрика — это порождающий паттерн проектирования, который используется для создания конкретных реализаций общего интерфейса.


## Пример

Автоматизируем ветеринарную клинику. Котов и собак лечим разными способами. Для этого создаем классы животных (`Cat` и `Dog`) с методом `sound`, который возвращает звук издаваемый здоровым животным.


### Проблема

**Наивная реализация.** В зависимости от параметра `type_of_animal` функции `get_animal`, хотим получить определенное животное (`Cat` или `Dog`).


In [2]:
class Cat:
    def sound(self) -> str:
        return "Мяу"


class Dog:
    def sound(self) -> str:
        return "Гав"

In [3]:
def get_animal(type_of_animal: str) -> object:
    """
    При добавлении нового класса, нужно будет модифицировать существующий код.
    Разработчикам нужно помнить все подобные места... такой код сложно поддерживать.
    """

    if type_of_animal == "Cat":
        return Cat()
    elif type_of_animal == "Dog":
        return Dog()
    else:
        raise NotImplementedError()


for selected_type in "Cat", "Dog":
    animal = get_animal(selected_type)
    print("Звук здорового животного:", animal.sound())

Звук здорового животного: Мяу
Звук здорового животного: Гав


### Решение

Добавляем абстрактный класс `AbsAnimal`, который предоставляет интерфейс животного. Классы `Cat` и `Dog` реализуют этот интерфейс. Получение объекта производится через метод `create_instance` фабричного класса `AnimalFactory`.


In [4]:
import abc


class AbsAnimal(abc.ABC):
    """
    Для добавления нового класса, достаточно отнаследоваться от этого абстрактного класса.
    """

    @abc.abstractmethod
    def sound(self) -> str:
        ...


class Cat(AbsAnimal):
    def sound(self) -> str:
        return "Мяу"


class Dog(AbsAnimal):
    def sound(self) -> str:
        return "Гав"

In [5]:
from typing import List, Type


class AnimalFactory:
    animals = (
        {}
    )  # После инициализации {'Cat': <class '__main__.Cat'>, 'Dog': <class '__main__.Dog'>}

    def __init__(self) -> None:
        self.load_animals()

    def get_all_subclasses(self, python_class: Type[object]) -> List[Type[object]]:
        """
        Метод возвращает все подклассы конкретного класса
        """
        python_class.__subclasses__()

        subclasses = set()
        check_these = [python_class]

        while check_these:
            parent = check_these.pop()
            for child in parent.__subclasses__():
                if child not in subclasses:
                    subclasses.add(child)
                    check_these.append(child)

        return list(subclasses)

    def load_animals(self) -> None:
        """
        Метод собирает все реализации `AbsAnimal` и сохраняет их в `self.animals`.
        """
        classes = self.get_all_subclasses(AbsAnimal)
        for selected_class in classes:
            self.animals.update({selected_class.__name__: selected_class})

    def create_instance(self, type_of_animal: str) -> AbsAnimal:
        """
        Метод возвращает объект типа `type_of_animal`.
        """
        if type_of_animal in self.animals:
            return self.animals[type_of_animal]()
        raise NotImplementedError()

In [6]:
factory = AnimalFactory()

for selected_type in "Cat", "Dog":
    animal = factory.create_instance(selected_type)
    print("Звук здорового животного:", animal.sound())

Звук здорового животного: Мяу
Звук здорового животного: Гав
